In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.impute import KNNImputer

In [3]:
import clean_data

## System 34

### Read data

In [4]:
pv_system_data_34 = pd.read_csv('data/ASYNC_annual_data_system_34.txt')

/tmp/ipykernel_16643/618913845.py:1: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  pv_system_data_34 = pd.read_csv('data/ASYNC_annual_data_system_34.txt')


In [5]:
pv_system_data_34.head()

,SiteID,Date-Time,ac_current,ac_power,ac_voltage,ambient_temp,dc_current,dc_power,dc_voltage,inverter_error_code,inverter_temp,module_temp,poa_irradiance,power_factor,relative_humidity,wind_direction,wind_speed
0,34,2010-08-23 11:13:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34,2010-08-23 11:15:00,117,100400,285,NaN,328,105100,317,0,58,NaN,NaN,-0.999,NaN,NaN,NaN
2,34,2010-08-23 11:17:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34,2010-08-23 11:18:07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34,2010-08-23 11:30:00,121,103100,284,NaN,338,108200,316,0,60,NaN,NaN,1,NaN,NaN,NaN


In [6]:
pv_system_data_34.shape

(345003, 17)

In [7]:
pv_system_data_34.columns.unique()

Index(['SiteID', 'Date-Time', 'ac_current', 'ac_power', 'ac_voltage',
       'ambient_temp', 'dc_current', 'dc_power', 'dc_voltage',
       'inverter_error_code', 'inverter_temp', 'module_temp', 'poa_irradiance',
       'power_factor', 'relative_humidity', 'wind_direction', 'wind_speed'],
      dtype='object')

### Delete additional headers

In [8]:
pv_system_data_34 = clean_data.delete_additional_header(pv_system_data_34)

In [9]:
assert((pv_system_data_34['SiteID'] == 'SiteID').any() == False)

### Rename columns 

In [10]:
pv_system_data_34 = pv_system_data_34.rename(columns = {'SiteID': 'site_id', 'Date-Time': 'timestamp'})

In [20]:
assert((pv_system_data_34['site_id'] == 'SiteID').any() == False)

### Convert data types

In [21]:
pv_system_data_34.dtypes

site_id                         int64
timestamp              datetime64[ns]
ac_current                    float64
ac_power                      float64
ac_voltage                    float64
ambient_temp                  float64
dc_current                    float64
dc_power                      float64
dc_voltage                    float64
inverter_error_code             int64
inverter_temp                 float64
module_temp                   float64
poa_irradiance                float64
power_factor                  float64
relative_humidity             float64
wind_direction                float64
wind_speed                    float64
dtype: object

In [22]:
pv_system_34_dtypes = {'site_id': int, 'timestamp': 'datetime64', 'ac_current': float, 'ac_power': float, 'ac_voltage': float, 'ambient_temp': float,
                       'dc_current': float, 'dc_power': float, 'dc_voltage': float, 'inverter_error_code': int, 'inverter_temp': float, 'module_temp': float,
                       'poa_irradiance': float, 'power_factor': float, 'relative_humidity': float, 'wind_direction': float, 'wind_speed': float}

In [23]:
pv_system_34_dtypes = clean_data.convert_data_types(pv_system_data_34, pv_system_34_dtypes)

In [29]:
assert(pv_system_data_34['timestamp'].dtype == 'datetime64[ns]')

### Missing data percentage

In [17]:
missing_data_percentage = clean_data.missing_values_percent(pv_system_data_34, 'timestamp')

In [18]:
missing_data_percentage

,site_id,timestamp,ac_current,ac_power,ac_voltage,ambient_temp,dc_current,dc_power,dc_voltage,inverter_error_code,inverter_temp,module_temp,poa_irradiance,power_factor,relative_humidity,wind_direction,wind_speed
timestamp,,,,,,,,,,,,,,,,,
2010,0.0,0.0,18.296178,18.296178,18.296178,1.321656,18.296178,18.296178,18.296178,0.0,18.296178,1.321656,1.321656,18.296178,1.321656,1.321656,1.321656
2011,0.0,0.0,0.037122,0.037122,0.037122,1.087950,0.037122,0.037122,0.037122,0.0,0.037122,1.087950,1.087950,0.037122,1.087950,1.087950,1.087950
2012,0.0,0.0,6.768907,6.768907,6.768907,0.031298,6.768907,6.768907,6.768907,0.0,6.768907,0.031298,0.031298,6.768907,0.031298,0.031298,0.031298
2013,0.0,0.0,0.313882,0.313882,0.313882,0.017121,0.313882,0.313882,0.313882,0.0,0.313882,0.017121,0.017121,0.313882,0.017121,0.017121,0.017121
2014,0.0,0.0,0.048533,0.048533,0.048533,0.548133,0.048533,0.048533,0.048533,0.0,0.048533,0.548133,0.548133,0.048533,0.548133,0.548133,0.548133
2015,0.0,0.0,0.000000,0.000000,0.000000,0.832261,0.000000,0.000000,0.000000,0.0,0.000000,0.832261,0.832261,0.000000,0.832261,0.832261,0.832261
2016,0.0,0.0,6.018782,6.018782,6.018782,0.836653,6.018782,6.018782,6.018782,0.0,6.018782,0.836653,0.836653,6.018782,0.836653,0.836653,0.836653
2017,0.0,0.0,0.014267,0.014267,0.014267,0.236838,0.014267,0.014267,0.014267,0.0,0.014267,0.236838,0.236838,0.014267,0.236838,0.236838,0.236838
2018,0.0,0.0,0.071465,0.071465,0.071465,15.462238,0.071465,0.071465,0.071465,0.0,0.071465,15.462238,15.462238,0.071465,15.462238,15.462238,15.462238
